In [2]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [5]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn_r50.py')

root='../../dataset/'

epoch = 'best_bbox_mAP_50_epoch_12'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = './work_dirs/faster_rcnn_r50'

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [6]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


In [7]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: ./work_dirs/faster_rcnn_r50/best_bbox_mAP_50_epoch_12.pth


In [8]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 10.4 task/s, elapsed: 469s, ETA:     0s

In [21]:
output[1]

[array([[5.9057135e+02, 6.7068665e+02, 8.9617572e+02, 9.3471631e+02,
         8.0826527e-01],
        [7.1101990e+02, 6.5292139e+02, 9.0729199e+02, 9.9708826e+02,
         5.9938478e-01],
        [3.5587451e+02, 6.5366107e+02, 8.8971814e+02, 9.5647394e+02,
         1.3118184e-01],
        [3.1908832e+02, 2.2373790e+02, 7.7152765e+02, 7.2929095e+02,
         1.1004272e-01],
        [7.9533966e+02, 6.8485522e+02, 8.9956256e+02, 1.0159348e+03,
         1.0638359e-01],
        [1.0183490e+02, 0.0000000e+00, 4.6662512e+02, 2.8404126e+02,
         7.8965254e-02],
        [6.8489026e+02, 6.6569885e+02, 7.9597607e+02, 9.3199780e+02,
         5.5990066e-02],
        [2.8441611e+02, 2.0044342e+02, 8.7785217e+02, 9.9377826e+02,
         5.2739274e-02]], dtype=float32),
 array([[7.174491e+02, 6.629227e+02, 9.097051e+02, 9.872744e+02,
         1.475105e-01]], dtype=float32),
 array([[3.1422156e+02, 2.8031287e+02, 7.6158276e+02, 7.0935040e+02,
         3.4106982e-01],
        [6.8274298e+02, 6.60151

In [7]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


,PredictionString,image_id
0,0 0.08096859 346.1726 609.7546 449.15417 742.0...,test/0000.jpg
1,0 0.80826527 590.57135 670.68665 896.1757 934....,test/0001.jpg
2,0 0.093713604 11.117355 256.00867 118.3358 495...,test/0002.jpg
3,0 0.20736967 30.023804 262.1125 916.0271 875.8...,test/0003.jpg
4,0 0.7361294 135.80121 299.60464 946.3022 789.6...,test/0004.jpg


In [17]:
df = pd.read_csv('./work_dirs/faster_rcnn_r50/submission_best_bbox_mAP_50_epoch_12.csv')
df.loc[1291]['PredictionString']

'0 0.19676936 455.00232 392.32516 838.4592 873.8369 9 0.050746284 494.17932 368.71875 816.05444 920.21594 '